<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
Aggregate xgboost classifier
    </font>
</h1>

In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|age|ستونی دارای مقادیر عددی که سن مشتری را نشان می‌دهد|
|job|ستونی با مقادیر دسته‌ای که وضعیت شغلی مشتری را نشان می‌دهد|
|marital|این ستون که مقادیر دسته‌ای را می‌پذیرد، وضعیت تاهل مشتری را مدل کرده است|
|education|وضعیت تحصیلی مشتری را به شکل دسته‌ای، نشان می‌دهد|
|default|نشان می‌دهد که آیا به صورت پیشفرض، مشتری اعتبار بانکی دارد یا خیر|
|housing|آیا مشتری برای مسکن، وتم گرفته است یا خیر|
|loan|آیا مشتری وام شخصی دریافت کرده است یا خیر؟|
|contact|نحوه ارتباط با مشتری را نشان می‌دهد. این ستون یکی از دو مقدار *cellular* و *telephone* را می‌تواند اختیار کند. به این معنی که بانک با تلفن همراه مشتری تماس گرفته است یا تلفن ثابت|
|month|این ستون دسته‌ای، نشان می‌دهد آخرین باری که با مشتری تماس گرفته شده است، در چه ماهی بوده است|
|day_of_week|روزی که آخرین تماس با مشتری گرفته شده است، چه روزی بوده است؟ (شنبه، یکشنبه، ...، جمعه)|
|campaign|تعداد تماس‌هایی که تا به حال با مشتری برای کمپین جاری گرفته شده است|
|pdays|تعداد روزی که از آخرین تماس با مشتری برای کمپین قبلی می‌گذرد. عدد ۹۹۹ به این معنی است که قبلا با مشتری تماس گرفته نشده است|
|previous|تعداد تماس‌هایی که با مشتری برای کمپین‌های قبلی گرفته شده است|
|poutcome|نتیجه کمپین‌های قبلی برای مشتری|
|emp.var.rate|نرخ تغییرات اشتغال در بازه سه ماهه|
|cons.price.idx|شاخص قیمت مصرف‌کننده در بازه یک ماهه|
|cons.conf.idx|شاخص اعتماد مصرف‌کننده در بازه یک ماهه|
|euribor3m|نرخ بهره بانکی برای وام‌های بین بانکی اروپایی|
|nr.employed|تعداد کارمندان در بازه سه ماهه|
|y|آیا فرد به پویش سرمایه‌گذاری بانک پاسخ مثبت داده یا خیر|
    
</font>
</div>
</center>


In [2]:
train_data = pd.read_csv('../data/bank_customers_train.csv')
test_data = pd.read_csv('../data/bank_customers_test.csv')

In [3]:
train_data.head(10)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,35,blue-collar,single,high.school,NaN,no,no,cellular,jul,thu,...,3,999,0,nonexistent,1.4,93.918,-42.7,4.958,5228.1,no
1,52,blue-collar,married,NaN,NaN,yes,no,telephone,jun,thu,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.961,5228.1,no
2,38,services,married,high.school,NaN,no,no,telephone,jun,tue,...,2,999,0,nonexistent,1.4,94.465,-41.8,4.864,5228.1,no
3,44,admin.,married,university.degree,no,yes,no,telephone,oct,wed,...,1,999,0,nonexistent,-3.4,92.431,-26.9,0.735,5017.5,yes
4,25,services,divorced,basic.4y,no,yes,no,telephone,may,fri,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.859,5191.0,no
5,49,services,married,high.school,no,no,no,cellular,aug,mon,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,no
6,30,technician,divorced,high.school,no,yes,yes,telephone,jun,tue,...,2,999,0,nonexistent,1.4,94.465,-41.8,4.961,5228.1,no
7,38,blue-collar,married,basic.6y,no,yes,no,cellular,may,mon,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.299,5099.1,no
8,34,admin.,married,high.school,no,yes,yes,cellular,jul,wed,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no
9,22,blue-collar,married,basic.9y,no,yes,no,cellular,may,thu,...,1,999,0,nonexistent,-1.8,92.893,-46.2,1.266,5099.1,no


In [4]:
train_data.shape

(39188, 21)

In [5]:
train_data.isna().sum()

age                  0
job                316
marital             79
education         1653
default           8162
housing            940
loan               940
contact              0
month                0
day_of_week          0
duration             0
campaign             0
pdays                0
previous             0
poutcome             0
emp.var.rate         0
cons.price.idx       0
cons.conf.idx        0
euribor3m            0
nr.employed          0
y                    0
dtype: int64

In [6]:
y = train_data['y']
train_data.drop(columns=['y'], inplace=True)


In [7]:
y = y.map({'no': 0, 'yes': 1})

In [8]:
train_data.drop(columns=['marital'])

,age,job,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,35,blue-collar,high.school,NaN,no,no,cellular,jul,thu,117,3,999,0,nonexistent,1.4,93.918,-42.7,4.958,5228.1
1,52,blue-collar,NaN,NaN,yes,no,telephone,jun,thu,118,4,999,0,nonexistent,1.4,94.465,-41.8,4.961,5228.1
2,38,services,high.school,NaN,no,no,telephone,jun,tue,253,2,999,0,nonexistent,1.4,94.465,-41.8,4.864,5228.1
3,44,admin.,university.degree,no,yes,no,telephone,oct,wed,749,1,999,0,nonexistent,-3.4,92.431,-26.9,0.735,5017.5
4,25,services,basic.4y,no,yes,no,telephone,may,fri,601,1,999,0,nonexistent,1.1,93.994,-36.4,4.859,5191.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39183,43,admin.,university.degree,no,no,no,cellular,may,tue,107,1,999,0,nonexistent,-1.8,92.893,-46.2,1.291,5099.1
39184,31,admin.,high.school,no,no,no,cellular,apr,fri,246,2,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1
39185,49,blue-collar,basic.4y,NaN,no,no,telephone,jun,fri,838,1,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1
39186,31,admin.,university.degree,no,yes,no,telephone,jun,wed,76,1,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1


In [9]:
def fill_dataset_with_modes(df):
    def fix_col(col):
        modes = col.mode(dropna=False)

        if len(modes) == 1 and pd.isna(modes.iloc[0]):
            return col

        if pd.isna(modes.iloc[0]):
            if len(modes) > 1:
                fill_value = modes.iloc[1]
            else:
                fill_value = modes.iloc[0]
        else:
            fill_value = modes.iloc[0]

        return col.fillna(fill_value)

    return df.apply(fix_col)


train_data_fill = fill_dataset_with_modes(train_data)
test_data_fill = fill_dataset_with_modes(test_data)


In [10]:
train_data_fill.head(3)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,35,blue-collar,single,high.school,no,no,no,cellular,jul,thu,117,3,999,0,nonexistent,1.4,93.918,-42.7,4.958,5228.1
1,52,blue-collar,married,university.degree,no,yes,no,telephone,jun,thu,118,4,999,0,nonexistent,1.4,94.465,-41.8,4.961,5228.1
2,38,services,married,high.school,no,no,no,telephone,jun,tue,253,2,999,0,nonexistent,1.4,94.465,-41.8,4.864,5228.1


In [11]:
train_data.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed'],
      dtype='object')

In [12]:
numeric_columns = [
    'age','duration', 'campaign', 'pdays',
       'previous', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed'
]

categorical_columns = [
    'poutcome','job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week'
]

st = StandardScaler()
train_part01 = st.fit_transform(train_data_fill[numeric_columns])
test_part01 = st.transform(test_data_fill[numeric_columns])

enc = OneHotEncoder(handle_unknown='ignore')
train_part02 = enc.fit_transform(train_data_fill[categorical_columns])
test_part02 = enc.transform(test_data_fill[categorical_columns])


In [13]:
from scipy import sparse

X = sparse.hstack([train_part01, train_part02], format='csr')
X_test = sparse.hstack([test_part01, test_part02], format='csr')


In [14]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42, test_size=0.33)

In [15]:
model = XGBClassifier(
    n_estimators=500,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='logloss'
)

model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [16]:
val_proba = model.predict_proba(X_val)[:, 1]
val_pred = (val_proba >= 0.5).astype(int)

print("Accuracy:", accuracy_score(y_val, val_pred))
print("F1:", f1_score(y_val, val_pred))
print("AUC:", roc_auc_score(y_val, val_proba))


Accuracy: 0.9173432304956314
F1: 0.5861401471157569
AUC: 0.9494015512361356


In [17]:
y_test = model.predict(X_test)
submission = pd.DataFrame(y_test, columns=['y'])
submission

,y
0,0
1,0
2,0
3,0
4,0
...,...
1995,0
1996,0
1997,0
1998,1
